In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import ast
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Load data
df = pd.read_csv(r'C:\Apple_leaf\Object_Detection.csv')
images, bboxes = [], []

# Load images and bounding boxes
for index, row in df.iterrows():
    img_array = np.array(ast.literal_eval(row['Image']), dtype=np.uint8)
    if img_array.size == 100 * 100 * 3:
        img_array = img_array.reshape(100, 100, 3)
        images.append(img_array)
        bboxes.append([row['x_min'], row['y_min'], row['x_max'], row['y_max']])
    else:
        print(f"Skipping index {index}: Image size is {img_array.size}, expected {100 * 100 * 3}")

# Convert images and bounding boxes to NumPy arrays
images = np.array(images).reshape(-1, 100, 100, 3)
bboxes = np.array(bboxes, dtype=np.float32)

# Train-test split
X_train, X_test, bbox_train, bbox_test = train_test_split(
    images, bboxes, test_size=0.2, random_state=42
)

# Define Standalone CNN Model
def build_cnn_model(input_shape=(100, 100, 3)):
    input_layer = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    bbox_output = tf.keras.layers.Dense(4, activation='linear')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=bbox_output, name="Standalone_CNN")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error'
    )
    return model

# Build and train the standalone CNN model
cnn_model = build_cnn_model()

# Train the CNN model
cnn_history = cnn_model.fit(
    X_train, bbox_train,
    validation_data=(X_test, bbox_test),
    epochs=25,
    batch_size=32
)

Skipping index 1620: Image size is 40000, expected 30000
Epoch 1/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 136ms/step - loss: 6214.9561 - val_loss: 901.9561
Epoch 2/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 140ms/step - loss: 907.6378 - val_loss: 891.0656
Epoch 3/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 144ms/step - loss: 861.3864 - val_loss: 919.7589
Epoch 4/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 135ms/step - loss: 865.1396 - val_loss: 939.7895
Epoch 5/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - loss: 862.2345 - val_loss: 929.9624
Epoch 6/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - loss: 852.8764 - val_loss: 925.7079
Epoch 7/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 143ms/step - loss: 795.0109 - val_loss: 957.4549
Epoch 8/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 139ms/step - loss: 773.0905 - val_loss: 947.3160
Epoch 9/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 153ms/step - loss: 738.2084 - val_loss: 961.1328
Epoch 10/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step - loss: 700.7260 - val_loss: 941.7839
Epoch 11/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s

In [7]:
# IoU Calculation
def calculate_iou(true_bbox, pred_bbox):
    x_min_true, y_min_true, x_max_true, y_max_true = true_bbox
    x_min_pred, y_min_pred, x_max_pred, y_max_pred = pred_bbox
    x_min_inter = max(x_min_true, x_min_pred)
    y_min_inter = max(y_min_true, y_min_pred)
    x_max_inter = min(x_max_true, x_max_pred)
    y_max_inter = min(y_max_true, y_max_pred)
    inter_area = max(0, x_max_inter - x_min_inter) * max(0, y_max_inter - y_min_inter)
    true_area = (x_max_true - x_min_true) * (y_max_true - y_min_true)
    pred_area = (x_max_pred - x_min_pred) * (y_max_pred - y_min_pred)
    union_area = true_area + pred_area - inter_area
    if union_area == 0:
        return 0.0
    iou = inter_area / union_area
    return iou

# Get predictions from the trained model
y_pred_bbox = cnn_model.predict(X_test)  # Corrected here

# IoU calculation on predictions
ious = [calculate_iou(true, pred) for true, pred in zip(bbox_test, y_pred_bbox)]
mean_iou = np.mean(ious)
print(f"Mean IoU for bounding box predictions from your model: {mean_iou:.4f}")

# Mean Squared Error and R2 Score
mse = mean_squared_error(bbox_test, y_pred_bbox)
r2 = r2_score(bbox_test, y_pred_bbox)
print(f"Mean Squared Error: {mse*100:.4f}%")
print(f"R2 Score: {r2*100:.4f}%")

# mAP Calculation (Mean Average Precision)
def calculate_map(ious, threshold=0.5):
    tp = sum([1 if iou >= threshold else 0 for iou in ious])
    fp = len(ious) - tp
    fn = 0  # We don't have true negatives in this case, just counting missed detections
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    return precision, recall

# Calculate mAP using IoUs
precision, recall = calculate_map(ious)
map_score = precision * recall  # Simplified mAP calculation, could be extended with averaging
print(f"Mean Average Precision (mAP): {map_score*100:.4f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
Mean IoU for bounding box predictions from transfer learning model: 69.8357
Mean Squared Error: 27.8654%
R2 Score: 72.8654%
Mean Average Precision (mAP): 70.5972%
